In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import re


In [2]:
# 2. action
#     + 行为个数, actionCnt
#     + 行为类型[1,2,3,4,5,6,7,8,9]的个数, mean，std，skew，kurt，var
#     + 行为类型的个数，actionTypeCnt
#     + 行为类型中浏览1次数的次数。
#     + 行为类型中浏览[2,3,4]次数， actionTypeIn24Cnt
#     + 行为类型中包含[5,6,7,8,9]的次数， actionTypeIn59Cnt
#     + 用户第一次和最后一个唤醒APP的差值（以天为单位），lastFirstDiff
#     + 用户最近一次唤醒APP的时间，[1对应的时间], actionTime1;-->[年，月，日，小时，周]
#     + 用户最近一次浏览APP的时间，[2,3,4对应的时间], actionTime24;-->[年，月，日，小时，周]
#     + 用户最近一次下单时间，[5-9对应的时间], actionTime59;-->[年，月，日，小时，周]
#     
#     + 计算平均时间的差值，这样就可以用到所有的时间信息，例如[2,3,4,6,8] 平均差 = (1+1+2+2) / 5
#     + -----------------------------------------------------------------------
#     + 用户第一次和最后一次使用APP的差值（以天为单位） / 唤醒的总次数，lastFirstDiffRate
#     + 对应的1，[2-4],[5-9]之间的差值和比例
#     + first, last: [2-4] - 1, [5-9] - [2,4]的差值，

In [3]:
df_train_data = pd.read_csv('../data/train/orderFuture_train.csv')
df_test_data = pd.read_csv('../data/test/orderFuture_test.csv')

# 首先选择action信息
df_train_action = pd.read_csv('../data/train/action_train.csv')
df_test_action = pd.read_csv('../data/test/action_test.csv')



In [4]:
# actionCnt
df_tmp = df_train_action.groupby(by=['userid','actionType'],axis=0,as_index=True).count()
df_tmp = df_tmp.unstack(fill_value=0)

# 没有想到好的处理方法，这里需要写入文件，手动删除处理。
# df_tmp.to_csv('../data/df_train_tmpCnt.csv')

# actionCnt
df_tmp = df_test_action.groupby(by=['userid','actionType'],axis=0,as_index=True).count()
df_tmp = df_tmp.unstack(fill_value=0)

# 没有想到好的处理方法，这里需要写入文件，手动删除处理。
# df_tmp.to_csv('../data/df_test_tmpCnt.csv')


In [5]:
# train

df_train_tmp = pd.read_csv('../data/df_train_tmpCnt.csv')
df_train_tmp.rename(columns={'1':'actionType1Cnt','2':'actionType2Cnt','3':'actionType3Cnt',
                       '4':'actionType4Cnt','5':'actionType5Cnt','6':'actionType6Cnt',
                       '7':'actionType7Cnt','8':'actionType8Cnt','9':'actionType9Cnt'},inplace=True)
df_train_data = pd.merge(df_train_data,df_train_tmp,how='left',on='userid')

type19 = ['actionType1Cnt','actionType2Cnt','actionType3Cnt',
          'actionType4Cnt','actionType5Cnt','actionType6Cnt',
          'actionType7Cnt','actionType8Cnt','actionType9Cnt']

# actionTypeCnt 存在的类型
df_train_data['actionTypeCnt'] = (df_train_data[type19] > 0).sum(axis=1)

# actionCnt 所有的类型的综合
df_train_data['actionCnt'] = df_train_data[type19].sum(axis=1)

# actionTypeIn24Cnt
df_train_data['actionTypeIn24Cnt'] = df_train_data[type19[1:4]].sum(axis=1)

# actionTypeIn59Cnt
df_train_data['actionTypeIn59Cnt'] = df_train_data[type19[4:]].sum(axis=1)

# actionTypeIn14Cnt
df_train_data['actionTypeIn14Cnt'] = df_train_data[type19[0:4]].sum(axis=1)

# actionTypeIn29Cnt
df_train_data['actionTypeIn29Cnt'] = df_train_data[type19[1:]].sum(axis=1)

# 每一个类型的点击率，占所有的比值的大小
df_train_data['actionType1Cnt_actionCnt_rate'] = df_train_data['actionType1Cnt'] / df_train_data['actionCnt']
df_train_data['actionType2Cnt_actionCnt_rate'] = df_train_data['actionType2Cnt'] / df_train_data['actionCnt']
df_train_data['actionType3Cnt_actionCnt_rate'] = df_train_data['actionType3Cnt'] / df_train_data['actionCnt']
df_train_data['actionType4Cnt_actionCnt_rate'] = df_train_data['actionType4Cnt'] / df_train_data['actionCnt']
df_train_data['actionType5Cnt_actionCnt_rate'] = df_train_data['actionType5Cnt'] / df_train_data['actionCnt']
df_train_data['actionType6Cnt_actionCnt_rate'] = df_train_data['actionType6Cnt'] / df_train_data['actionCnt']
df_train_data['actionType7Cnt_actionCnt_rate'] = df_train_data['actionType7Cnt'] / df_train_data['actionCnt']
df_train_data['actionType8Cnt_actionCnt_rate'] = df_train_data['actionType8Cnt'] / df_train_data['actionCnt']
df_train_data['actionType9Cnt_actionCnt_rate'] = df_train_data['actionType9Cnt'] / df_train_data['actionCnt']

df_train_data['actionTypeCnt_actionCnt_rate'] = df_train_data['actionTypeCnt'] / df_train_data['actionCnt']
df_train_data['actionTypeIn24Cnt_actionCnt_rate'] = df_train_data['actionTypeIn24Cnt'] / df_train_data['actionCnt']
df_train_data['actionTypeIn59Cnt_actionCnt_rate'] = df_train_data['actionTypeIn59Cnt'] / df_train_data['actionCnt']
df_train_data['actionTypeIn14Cnt_actionCnt_rate'] = df_train_data['actionTypeIn14Cnt'] / df_train_data['actionCnt']
df_train_data['actionTypeIn29Cnt_actionCnt_rate'] = df_train_data['actionTypeIn29Cnt'] / df_train_data['actionCnt']



In [6]:
# test

df_test_tmp = pd.read_csv('../data/df_test_tmpCnt.csv')
df_test_tmp.rename(columns={'1':'actionType1Cnt','2':'actionType2Cnt','3':'actionType3Cnt',
                       '4':'actionType4Cnt','5':'actionType5Cnt','6':'actionType6Cnt',
                       '7':'actionType7Cnt','8':'actionType8Cnt','9':'actionType9Cnt'},inplace=True)
df_test_data = pd.merge(df_test_data,df_test_tmp,how='left',on='userid')

type19 = ['actionType1Cnt','actionType2Cnt','actionType3Cnt',
          'actionType4Cnt','actionType5Cnt','actionType6Cnt',
          'actionType7Cnt','actionType8Cnt','actionType9Cnt']

# actionTypeCnt 存在的类型
df_test_data['actionTypeCnt'] = (df_test_data[type19] > 0).sum(axis=1)

# actionCnt 所有的类型的综合
df_test_data['actionCnt'] = df_test_data[type19].sum(axis=1)

# actionTypeIn24Cnt
df_test_data['actionTypeIn24Cnt'] = df_test_data[type19[1:4]].sum(axis=1)

# actionTypeIn59Cnt
df_test_data['actionTypeIn59Cnt'] = df_test_data[type19[4:]].sum(axis=1)

# actionTypeIn14Cnt
df_test_data['actionTypeIn14Cnt'] = df_test_data[type19[0:4]].sum(axis=1)

# actionTypeIn29Cnt
df_test_data['actionTypeIn29Cnt'] = df_test_data[type19[1:]].sum(axis=1)

# 每一个类型的点击率，占所有的比值的大小
df_test_data['actionType1Cnt_actionCnt_rate'] = df_test_data['actionType1Cnt'] / df_test_data['actionCnt']
df_test_data['actionType2Cnt_actionCnt_rate'] = df_test_data['actionType2Cnt'] / df_test_data['actionCnt']
df_test_data['actionType3Cnt_actionCnt_rate'] = df_test_data['actionType3Cnt'] / df_test_data['actionCnt']
df_test_data['actionType4Cnt_actionCnt_rate'] = df_test_data['actionType4Cnt'] / df_test_data['actionCnt']
df_test_data['actionType5Cnt_actionCnt_rate'] = df_test_data['actionType5Cnt'] / df_test_data['actionCnt']
df_test_data['actionType6Cnt_actionCnt_rate'] = df_test_data['actionType6Cnt'] / df_test_data['actionCnt']
df_test_data['actionType7Cnt_actionCnt_rate'] = df_test_data['actionType7Cnt'] / df_test_data['actionCnt']
df_test_data['actionType8Cnt_actionCnt_rate'] = df_test_data['actionType8Cnt'] / df_test_data['actionCnt']
df_test_data['actionType9Cnt_actionCnt_rate'] = df_test_data['actionType9Cnt'] / df_test_data['actionCnt']

df_test_data['actionTypeCnt_actionCnt_rate'] = df_test_data['actionTypeCnt'] / df_test_data['actionCnt']
df_test_data['actionTypeIn24Cnt_actionCnt_rate'] = df_test_data['actionTypeIn24Cnt'] / df_test_data['actionCnt']
df_test_data['actionTypeIn59Cnt_actionCnt_rate'] = df_test_data['actionTypeIn59Cnt'] / df_test_data['actionCnt']
df_test_data['actionTypeIn14Cnt_actionCnt_rate'] = df_test_data['actionTypeIn14Cnt'] / df_test_data['actionCnt']
df_test_data['actionTypeIn29Cnt_actionCnt_rate'] = df_test_data['actionTypeIn29Cnt'] / df_test_data['actionCnt']



In [7]:
## train
userlist = list(set(df_train_action['userid']))

column = ['userid','actionTimeFirst','actionTimeLast','actionTime1First','actionTime1Last',
          'actionTime24First','actionTime24Last','actionTime59First','actionTime59Last']

df_train_time = pd.DataFrame(columns=column)
k = 0
for user in userlist:
    k+=1
    if k % 1000 == 0:
        print(k)
    
    df_t = df_train_action[df_train_action['userid'] == user]
    df_t.sort_values('actionTime',inplace=True)
    
    timedata = [user]
    
    # timeFeature
    # 第一次操作APP，最后一个操作APP时间
    if len(df_t) > 0:
        timedata.append(df_t['actionTime'].values[0])
        timedata.append(df_t['actionTime'].values[-1])
    else:
        timedata.append(np.nan)
        timedata.append(np.nan)
    
    # 1 对应的第一次和最后一个时间
    if len(df_t[df_t['actionType']==1]>0):
        timedata.append(df_t[df_t['actionType']==1]['actionTime'].values[0])
        timedata.append(df_t[df_t['actionType']==1]['actionTime'].values[-1])
    else:
        timedata.append(np.nan)
        timedata.append(np.nan)
    
    # 2-4 对应的第一次和最后一个时间
    if len(df_t[df_t['actionType'].isin([2,3,4])])>0:
        timedata.append(df_t[df_t['actionType'].isin([2,3,4])]['actionTime'].values[0])
        timedata.append(df_t[df_t['actionType'].isin([2,3,4])]['actionTime'].values[-1])
    else:
        timedata.extend([np.nan,np.nan])

    # 5-9 对应的第一次和最后一个时间
    if len(df_t[df_t['actionType'].isin([5,6,7,8,9])])>0:
        timedata.append(df_t[df_t['actionType'].isin([5,6,7,8,9])]['actionTime'].values[0])
        timedata.append(df_t[df_t['actionType'].isin([5,6,7,8,9])]['actionTime'].values[-1])
    else:
        timedata.append(np.nan)
        timedata.append(np.nan)
    
    df_train_time = df_train_time.append(pd.DataFrame([timedata],columns=column),ignore_index=True)

df_train_data = pd.merge(df_train_data,df_train_time,how='left',on='userid')


/home/xiaoran/Tools/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000


In [10]:
# 使用2018-01-01 00:00:00 填充空值,对应的时间戳： 1514736000
df_train_data[column[1:]] = df_train_data[column[1:]].fillna(1514736000)

# In[16]:

df_time_last = pd.DataFrame()
df_time_last['curTime'] = [1514736000 for i in range(len(df_train_data))]

df_train_data['actionTime1CurLastDiff'] = df_time_last['curTime']-df_train_data['actionTime1Last']
df_train_data['actionTime24CurLastDiff'] = df_time_last['curTime']-df_train_data['actionTime24Last']  
df_train_data['actionTime59CurLastDiff'] = df_time_last['curTime']-df_train_data['actionTime59Last']  
df_train_data['actionTime1CurFirstDiff'] = df_time_last['curTime']-df_train_data['actionTime1First']
df_train_data['actionTime24CurFirstDiff'] = df_time_last['curTime']-df_train_data['actionTime24First']  
df_train_data['actionTime59CurFirstDiff'] = df_time_last['curTime']-df_train_data['actionTime59First']

# In[17]:

# 计算时间差和[年月日，小时，，分钟，秒，周]
df_train_data['actionTimeLastFirstDiff'] = df_train_data['actionTimeLast']-df_train_data['actionTimeFirst']
df_train_data['actionTime1LastFirstDiff'] = df_train_data['actionTime1Last']-df_train_data['actionTime1First']
df_train_data['actionTime24LastFirstDiff'] = df_train_data['actionTime24Last']-df_train_data['actionTime24First']
df_train_data['actionTime59LastFirstDiff'] = df_train_data['actionTime59Last']-df_train_data['actionTime59First']

##########
df_train_data['actionTime24LastactiomTime1LastDiff'] = df_train_data['actionTime24Last']-df_train_data['actionTime1Last']
df_train_data['actionTime24FirstactiomTime1FirstDiff'] = df_train_data['actionTime24First']-df_train_data['actionTime1First']

df_train_data['actionTime59LastactiomTime24LastDiff'] = df_train_data['actionTime59Last']-df_train_data['actionTime24Last']
df_train_data['actionTime59FirstactiomTime24FirstDiff'] = df_train_data['actionTime59First']-df_train_data['actionTime24First']

df_train_data['actionTimeLastFirstDiffRate'] = df_train_data['actionTimeLastFirstDiff'] / df_train_data['actionCnt']
df_train_data['actionTime1LastFirstDiffRate'] = df_train_data['actionTime1LastFirstDiff'] / df_train_data['actionType1Cnt']
df_train_data['actionTime24LastFirstDiffRate'] = df_train_data['actionTime24LastFirstDiff'] / df_train_data['actionTypeIn24Cnt']
df_train_data['actionTime59LastFirstDiffRate'] = df_train_data['actionTime59LastFirstDiff'] / df_train_data['actionTypeIn59Cnt']


# 替换无穷大的值
df_train_data['actionTimeLastFirstDiffRate'].replace([np.inf, -np.inf], 0 ,inplace=True)
df_train_data['actionTime1LastFirstDiffRate'].replace([np.inf, -np.inf], 0 ,inplace=True)
df_train_data['actionTime24LastFirstDiffRate'].replace([np.inf, -np.inf], 0 ,inplace=True)
df_train_data['actionTime59LastFirstDiffRate'].replace([np.inf, -np.inf], 0 ,inplace=True)


# 替换无穷大的值
df_train_data['actionTimeLastFirstDiffRate'].fillna(0,inplace=True)
df_train_data['actionTime1LastFirstDiffRate'].fillna(0,inplace=True)
df_train_data['actionTime24LastFirstDiffRate'].fillna(0,inplace=True)
df_train_data['actionTime59LastFirstDiffRate'].fillna(0,inplace=True)


In [11]:
## test
userlist = list(set(df_test_action['userid']))

column = ['userid','actionTimeFirst','actionTimeLast','actionTime1First','actionTime1Last',
          'actionTime24First','actionTime24Last','actionTime59First','actionTime59Last']

df_test_time = pd.DataFrame(columns=column)
k = 0
for user in userlist:
    k+=1
    if k % 1000 == 0:
        print(k)
    
    df_t = df_test_action[df_test_action['userid'] == user]
    df_t.sort_values('actionTime',inplace=True)
    
    timedata = [user]
    
    # timeFeature
    # 第一次操作APP，最后一个操作APP时间
    if len(df_t) > 0:
        timedata.append(df_t['actionTime'].values[0])
        timedata.append(df_t['actionTime'].values[-1])
    else:
        timedata.append(np.nan)
        timedata.append(np.nan)
    
    # 1 对应的第一次和最后一个时间
    if len(df_t[df_t['actionType']==1]>0):
        timedata.append(df_t[df_t['actionType']==1]['actionTime'].values[0])
        timedata.append(df_t[df_t['actionType']==1]['actionTime'].values[-1])
    else:
        timedata.append(np.nan)
        timedata.append(np.nan)
    
    # 2-4 对应的第一次和最后一个时间
    if len(df_t[df_t['actionType'].isin([2,3,4])])>0:
        timedata.append(df_t[df_t['actionType'].isin([2,3,4])]['actionTime'].values[0])
        timedata.append(df_t[df_t['actionType'].isin([2,3,4])]['actionTime'].values[-1])
    else:
        timedata.extend([np.nan,np.nan])

    # 5-9 对应的第一次和最后一个时间
    if len(df_t[df_t['actionType'].isin([5,6,7,8,9])])>0:
        timedata.append(df_t[df_t['actionType'].isin([5,6,7,8,9])]['actionTime'].values[0])
        timedata.append(df_t[df_t['actionType'].isin([5,6,7,8,9])]['actionTime'].values[-1])
    else:
        timedata.append(np.nan)
        timedata.append(np.nan)
    
    df_test_time = df_test_time.append(pd.DataFrame([timedata],columns=column),ignore_index=True)

df_test_data = pd.merge(df_test_data,df_test_time,how='left',on='userid')


/home/xiaoran/Tools/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [12]:
# 使用2018-01-01 00:00:00 填充空值,对应的时间戳： 1514736000
df_test_data[column[1:]] = df_test_data[column[1:]].fillna(1514736000)


# In[16]:

df_time_last = pd.DataFrame()
df_time_last['curTime'] = [1514736000 for i in range(len(df_test_data))]


df_test_data['actionTime1CurLastDiff'] = df_time_last['curTime']-df_test_data['actionTime1Last']
df_test_data['actionTime24CurLastDiff'] = df_time_last['curTime']-df_test_data['actionTime24Last'] 
df_test_data['actionTime59CurLastDiff'] = df_time_last['curTime']-df_test_data['actionTime59Last'] 

df_test_data['actionTime1CurFirstDiff'] = df_time_last['curTime']-df_test_data['actionTime1First']
df_test_data['actionTime24CurFirstDiff'] = df_time_last['curTime']-df_test_data['actionTime24First']
df_test_data['actionTime59CurFirstDiff'] = df_time_last['curTime']-df_test_data['actionTime59First'] 

# In[17]:

# 计算时间差和[年月日，小时，，分钟，秒，周]
df_test_data['actionTimeLastFirstDiff'] = df_test_data['actionTimeLast']-df_test_data['actionTimeFirst']
df_test_data['actionTime1LastFirstDiff'] = df_test_data['actionTime1Last']-df_test_data['actionTime1First']
df_test_data['actionTime24LastFirstDiff'] = df_test_data['actionTime24Last']-df_test_data['actionTime24First']
df_test_data['actionTime59LastFirstDiff'] = df_test_data['actionTime59Last']-df_test_data['actionTime59First'] 

##########
df_test_data['actionTime24LastactiomTime1LastDiff'] = df_test_data['actionTime24Last']-df_test_data['actionTime1Last']  
df_test_data['actionTime24FirstactiomTime1FirstDiff'] = df_test_data['actionTime24First']-df_test_data['actionTime1First']  

df_test_data['actionTime59LastactiomTime24LastDiff'] = df_test_data['actionTime59Last']-df_test_data['actionTime24Last']  
df_test_data['actionTime59FirstactiomTime24FirstDiff'] = df_test_data['actionTime59First']-df_test_data['actionTime24First']  

df_test_data['actionTimeLastFirstDiffRate'] = df_test_data['actionTimeLastFirstDiff'] / df_test_data['actionCnt']
df_test_data['actionTime1LastFirstDiffRate'] = df_test_data['actionTime1LastFirstDiff'] / df_test_data['actionType1Cnt']
df_test_data['actionTime24LastFirstDiffRate'] = df_test_data['actionTime24LastFirstDiff'] / df_test_data['actionTypeIn24Cnt']
df_test_data['actionTime59LastFirstDiffRate'] = df_test_data['actionTime59LastFirstDiff'] / df_test_data['actionTypeIn59Cnt']


# 替换无穷大的值
df_test_data['actionTimeLastFirstDiffRate'].replace([np.inf, -np.inf], 0 ,inplace=True)
df_test_data['actionTime1LastFirstDiffRate'].replace([np.inf, -np.inf], 0 ,inplace=True)
df_test_data['actionTime24LastFirstDiffRate'].replace([np.inf, -np.inf], 0 ,inplace=True)
df_test_data['actionTime59LastFirstDiffRate'].replace([np.inf, -np.inf], 0 ,inplace=True)


# 替换无穷大的值
df_test_data['actionTimeLastFirstDiffRate'].fillna(0,inplace=True)
df_test_data['actionTime1LastFirstDiffRate'].fillna(0,inplace=True)
df_test_data['actionTime24LastFirstDiffRate'].fillna(0,inplace=True)
df_test_data['actionTime59LastFirstDiffRate'].fillna(0,inplace=True)


In [13]:
df_train_data.info()
df_test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40307 entries, 0 to 40306
Data columns (total 57 columns):
userid                                    40307 non-null object
orderType                                 40307 non-null int64
actionType1Cnt                            40307 non-null int64
actionType2Cnt                            40307 non-null int64
actionType3Cnt                            40307 non-null int64
actionType4Cnt                            40307 non-null int64
actionType5Cnt                            40307 non-null int64
actionType6Cnt                            40307 non-null int64
actionType7Cnt                            40307 non-null int64
actionType8Cnt                            40307 non-null int64
actionType9Cnt                            40307 non-null int64
actionTypeCnt                             40307 non-null int64
actionCnt                                 40307 non-null int64
actionTypeIn24Cnt                         40307 non-null int64
actionTy

In [14]:
column

['userid',
 'actionTimeFirst',
 'actionTimeLast',
 'actionTime1First',
 'actionTime1Last',
 'actionTime24First',
 'actionTime24Last',
 'actionTime59First',
 'actionTime59Last']

In [15]:
for f in column[1:]:
    df_train_data.drop(f,axis=1,inplace=True)
    df_test_data.drop(f,axis=1,inplace=True)


In [16]:
df_train_data.drop('orderType',axis=1,inplace=True)
df_train_data.to_csv('../data/dataSet/df_train_6.csv',index=False,index_label=False)
df_test_data.to_csv('../data/dataSet/df_test_6.csv',index=False,index_label=False)
